# Importing Libraries

In [118]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD

# Loading the Data

In [119]:
df_bg = pd.read_csv('../datasets/games-cleaned.csv')
df_rv = pd.read_csv('../datasets/reviews-cleaned.csv')

# Model-Based Recommenders

In [120]:
ratings_matrix = df_rv.pivot_table(index='user', columns='name', values='rating').fillna(0)
ratings_matrix.head()

name,"...and then, we held hands.",...und tschüss!,10 Days in Africa,10 Days in Asia,10 Days in Europe,10 Days in the Americas,10 Days in the USA,10 Minute Heist: The Wizard's Tower,10' to Kill,1000 Blank White Cards,...,Zooloretto Junior,Zooloretto: The Dice Game,Zoowaboo,Zug um Zug: Deutschland,Zulus on the Ramparts!: The Battle of Rorke's Drift – Second Edition,ZÈRTZ,[redacted],duck! duck! Go!,ebbes,iKNOW
user,,,,,,,,,,,,,,,,,,,,,
1 Family Meeple,0.0,0.0,0.0,0.0,4.1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.5,4.0,0.0,0.0
1000rpm,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28green,0.0,0.0,0.0,0.0,7.0,0.0,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42amu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
549sd,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0


In [121]:
user_ratings_mean = np.mean(ratings_matrix, axis = 1)
user_ratings_mean

user
1 Family Meeple    0.826877
1000rpm            0.776735
28green            0.936381
42amu              0.921868
549sd              1.191510
                     ...   
zsknight           0.795275
zumba              0.842863
zumikon            1.076822
zunnesteke         0.826931
zuzusdad           0.965561
Length: 3067, dtype: float64

In [122]:
# R_min = (ratings_matrix - user_ratings_mean.values.reshape(-1, 1)).min().min()
# R_demeaned =  + ratings_matrix - user_ratings_mean.values.reshape(-1, 1) - R_min
R_demeaned = ratings_matrix - user_ratings_mean.values.reshape(-1, 1)

R_demeaned.head()

name,"...and then, we held hands.",...und tschüss!,10 Days in Africa,10 Days in Asia,10 Days in Europe,10 Days in the Americas,10 Days in the USA,10 Minute Heist: The Wizard's Tower,10' to Kill,1000 Blank White Cards,...,Zooloretto Junior,Zooloretto: The Dice Game,Zoowaboo,Zug um Zug: Deutschland,Zulus on the Ramparts!: The Battle of Rorke's Drift – Second Edition,ZÈRTZ,[redacted],duck! duck! Go!,ebbes,iKNOW
user,,,,,,,,,,,,,,,,,,,,,
1 Family Meeple,-0.826877,-0.826877,-0.826877,-0.826877,3.273123,-0.826877,-0.826877,-0.826877,-0.826877,-0.826877,...,-0.826877,-0.826877,-0.826877,-0.826877,-0.826877,-0.826877,5.673123,3.173123,-0.826877,-0.826877
1000rpm,-0.776735,-0.776735,-0.776735,-0.776735,6.223265,-0.776735,-0.776735,-0.776735,-0.776735,-0.776735,...,-0.776735,6.223265,-0.776735,-0.776735,-0.776735,-0.776735,-0.776735,-0.776735,-0.776735,-0.776735
28green,-0.936381,-0.936381,-0.936381,-0.936381,6.063619,-0.936381,6.063619,-0.936381,-0.936381,-0.936381,...,-0.936381,-0.936381,-0.936381,-0.936381,-0.936381,-0.936381,-0.936381,-0.936381,-0.936381,-0.936381
42amu,-0.921868,-0.921868,-0.921868,-0.921868,-0.921868,-0.921868,-0.921868,-0.921868,7.078132,-0.921868,...,-0.921868,-0.921868,-0.921868,-0.921868,-0.921868,-0.921868,-0.921868,-0.921868,-0.921868,-0.921868
549sd,-1.191510,-1.191510,-1.191510,-1.191510,-1.191510,-1.191510,4.808490,-1.191510,-1.191510,-1.191510,...,-1.191510,-1.191510,-1.191510,-1.191510,-1.191510,-1.191510,3.808490,-1.191510,-1.191510,-1.191510


In [133]:
# Convert the pivot table to a sparse matrix
matrix = csr_matrix(R_demeaned.values)

# Apply TruncatedSVD to the matrix
svd = TruncatedSVD(n_components=10)
latent_features = svd.fit_transform(matrix)

# Use the latent features to recommend games to a user
user_index = 0
user_latent_features = latent_features[user_index, :]

# Calculate the dot product between the user's latent features and all latent features
similarity = latent_features.dot(user_latent_features)

# Sort the games by similarity and recommend the most similar games
most_similar_games = R_demeaned.columns[similarity.argsort()[::-1]].to_list()

print(f'Recommended games for {R_demeaned.index[user_index]}: \n{most_similar_games[:10]}')

Recommended games for user 1 Family Meeple: 
['Aeroplanes: Aviation Ascendant', "King's Breakfast", 'Quartz', 'Cockroach Poker', 'D-Day at Omaha Beach', 'Diamonds Club', 'Advanced Squad Leader: Starter Kit #1', 'Quarriors!', 'B-17 Flying Fortress Leader', 'Monumental']
